In [2]:
from Distillation_disc_cont import simulator
from Actor_Critic_lstm import Agent
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
import dill

In [8]:
#dill.load_session('Agent.db')

In [3]:
%autoreload

In [4]:
env = simulator()
agent = Agent(env = env, alpha = 1e-6, beta = 1e-7, gamma = 0.99)
agent.actor.summary()
#agent.critic.summary()
#agent.policy.summary()

W0930 16:59:15.710533  5008 deprecation_wrapper.py:119] From C:\Users\meatrobot\Anaconda3\envs\RL\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0930 16:59:15.730479  5008 deprecation_wrapper.py:119] From C:\Users\meatrobot\Anaconda3\envs\RL\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0930 16:59:15.735663  5008 deprecation_wrapper.py:119] From C:\Users\meatrobot\Anaconda3\envs\RL\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

D:\Github\RL-Process-Design\Discrete and Continuous\Actor_Critic_lstm.py:59: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  actor = Model(input = [input, delta], output = [policy_discrete_probs, polic

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 6)            0                                            
__________________________________________________________________________________________________
fc1 (Dense)                     (None, 20)           140         Input[0][0]                      
__________________________________________________________________________________________________
fc2 (Dense)                     (None, 20)           420         fc1[0][0]                        
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 20, 1)        0           fc2[0][0]                        
____________________________________________________________________________________________

D:\Github\RL-Process-Design\Discrete and Continuous\Actor_Critic_lstm.py:62: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  critic = Model(input = [input], output = [Values])
D:\Github\RL-Process-Design\Discrete and Continuous\Actor_Critic_lstm.py:67: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  policy = Model(input = [input], output = [policy_discrete_probs, policy_continuous])


In [ ]:
score_history = []
num_episodes = int(1e4)
for i in range(num_episodes):
	done = False
	
	state = env.reset()
	score = 0
	
	while not done:
		action = agent.choose_action_epsgreedy(state, i, num_episodes)
		next_state, reward, done, info = env.step(action)
		agent.learn(state,  action, reward, next_state, done)
		state = next_state
		score += reward
	
	score_history.append(score)
	avg_score = np.mean(score_history[-100:]) #average of last 100 scores
	if i%100 == 0:
		print(f'episode {i}, avg_score {avg_score}, last action {action}')
        


W0930 16:59:49.022184  5008 deprecation_wrapper.py:119] From C:\Users\meatrobot\Anaconda3\envs\RL\lib\site-packages\keras\backend\tensorflow_backend.py:3005: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0930 16:59:49.567752  5008 deprecation.py:323] From C:\Users\meatrobot\Anaconda3\envs\RL\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
D:\Github\RL-Process-Design\Discrete and Continuous\Distillation_disc_cont.py:63: RuntimeWarning: invalid value encountered in double_scalars
  LK_D = tops[Light_Key]/sum(tops)


episode 0, avg_score -12507.20345099872, last action (2, array([[0.94902276]]))


D:\Github\RL-Process-Design\Discrete and Continuous\Distillation_disc_cont.py:64: RuntimeWarning: invalid value encountered in double_scalars
  LK_B = bots[Light_Key]/sum(bots)
D:\Github\RL-Process-Design\Discrete and Continuous\Distillation_disc_cont.py:74: RuntimeWarning: invalid value encountered in log
  N =  np.log(LK_D/(1-LK_D) * (1-LK_B)/LK_B)/np.log(self.relative_volatility[Light_Key])


episode 100, avg_score -6018.9484413725695, last action (2, array([[0.999]], dtype=float32))
episode 200, avg_score -9189.837547078818, last action (2, array([[0.16456735]]))
episode 300, avg_score -10680.380330005339, last action (2, array([[-0.6109293]]))
episode 400, avg_score -6542.204614947083, last action (2, array([[0.52066563]]))
episode 500, avg_score -10125.22753459931, last action (2, array([[0.75844605]]))
episode 600, avg_score -12590.265065797605, last action (2, array([[0.999]], dtype=float32))
episode 700, avg_score -11802.435072919778, last action (0, array([[0.22936471]]))
episode 800, avg_score -11944.083857722835, last action (2, array([[0.999]], dtype=float32))


In [ ]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N

In [ ]:
episodes = np.arange(num_episodes)
smoothed_rews = running_mean(score_history, 10)
plt.plot(episodes[-len(smoothed_rews):], smoothed_rews)
plt.plot(episodes, score_history,color='grey', alpha=0.3)
plt.xlabel("steps")
plt.ylabel("reward")
plt.legend(["avg reward", "reward"])

In [ ]:
#test agent
state = env.reset()
done = False
while done == False:
    state = state[np.newaxis, :]
    action_discrete, action_continuous = agent.policy.predict(state)
    action_discrete = np.argmax(action_discrete)
    action = action_discrete, action_continuous
    
    state, reward, done, info = env.step(action)
    print(f'action LK: {action_discrete}, action split: {action_continuous[0]}, reward: {reward}')

In [ ]:
env.stream_table

In [ ]:
action_discrete, action_continuous = agent.policy.predict(state[np.newaxis, :])

In [ ]:
#dill.dump_session('Agent.db')